## Preprocessing

In [1]:
# Import our dependencies
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import pandas as pd
import tensorflow as tf
import keras_tuner as kt 

#  Import and read the charity_data.csv.
import pandas as pd 
application_df = pd.read_csv("https://static.bc-edx.com/data/dl-1-2/m21/lms/starter/charity_data.csv")
application_df.head()

2023-09-24 17:44:28.334285: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


Using TensorFlow backend


,EIN,NAME,APPLICATION_TYPE,AFFILIATION,CLASSIFICATION,USE_CASE,ORGANIZATION,STATUS,INCOME_AMT,SPECIAL_CONSIDERATIONS,ASK_AMT,IS_SUCCESSFUL
0,10520599,BLUE KNIGHTS MOTORCYCLE CLUB,T10,Independent,C1000,ProductDev,Association,1,0,N,5000,1
1,10531628,AMERICAN CHESAPEAKE CLUB CHARITABLE TR,T3,Independent,C2000,Preservation,Co-operative,1,1-9999,N,108590,1
2,10547893,ST CLOUD PROFESSIONAL FIREFIGHTERS,T5,CompanySponsored,C3000,ProductDev,Association,1,0,N,5000,0
3,10553066,SOUTHSIDE ATHLETIC ASSOCIATION,T3,CompanySponsored,C2000,Preservation,Trust,1,10000-24999,N,6692,1
4,10556103,GENETIC RESEARCH INSTITUTE OF THE DESERT,T3,Independent,C1000,Heathcare,Trust,1,100000-499999,N,142590,1


In [2]:
# Drop the non-beneficial ID columns, 'EIN' and 'NAME'.
application_df= application_df.drop (["EIN","NAME"], axis=1)
application_df

,APPLICATION_TYPE,AFFILIATION,CLASSIFICATION,USE_CASE,ORGANIZATION,STATUS,INCOME_AMT,SPECIAL_CONSIDERATIONS,ASK_AMT,IS_SUCCESSFUL
0,T10,Independent,C1000,ProductDev,Association,1,0,N,5000,1
1,T3,Independent,C2000,Preservation,Co-operative,1,1-9999,N,108590,1
2,T5,CompanySponsored,C3000,ProductDev,Association,1,0,N,5000,0
3,T3,CompanySponsored,C2000,Preservation,Trust,1,10000-24999,N,6692,1
4,T3,Independent,C1000,Heathcare,Trust,1,100000-499999,N,142590,1
...,...,...,...,...,...,...,...,...,...,...
34294,T4,Independent,C1000,ProductDev,Association,1,0,N,5000,0
34295,T4,CompanySponsored,C3000,ProductDev,Association,1,0,N,5000,0
34296,T3,CompanySponsored,C2000,Preservation,Association,1,0,N,5000,0
34297,T5,Independent,C3000,ProductDev,Association,1,0,N,5000,1


In [3]:
# Determine the number of unique values in each column.
application_df.nunique()

APPLICATION_TYPE            17
AFFILIATION                  6
CLASSIFICATION              71
USE_CASE                     5
ORGANIZATION                 4
STATUS                       2
INCOME_AMT                   9
SPECIAL_CONSIDERATIONS       2
ASK_AMT                   8747
IS_SUCCESSFUL                2
dtype: int64

In [4]:
# Look at APPLICATION_TYPE value counts for binning
application_counts = application_df['APPLICATION_TYPE'].value_counts()
application_counts

T3     27037
T4      1542
T6      1216
T5      1173
T19     1065
T8       737
T7       725
T10      528
T9       156
T13       66
T12       27
T2        16
T25        3
T14        3
T29        2
T15        2
T17        1
Name: APPLICATION_TYPE, dtype: int64

In [5]:
# Choose a cutoff value and create a list of application types to be replaced
# use the variable name `application_types_to_replace`
application_types_to_replace = list(application_counts[application_counts < 1000].index)

# Replace in dataframe
for app in application_types_to_replace:
    application_df['APPLICATION_TYPE'] = application_df['APPLICATION_TYPE'].replace(app,"Other")

# Check to make sure binning was successful
application_df['APPLICATION_TYPE'].value_counts()

T3       27037
Other     2266
T4        1542
T6        1216
T5        1173
T19       1065
Name: APPLICATION_TYPE, dtype: int64

In [6]:
# Look at CLASSIFICATION value counts for binning
classification_counts = application_df['CLASSIFICATION'].value_counts()
classification_counts

C1000    17326
C2000     6074
C1200     4837
C3000     1918
C2100     1883
         ...  
C4120        1
C8210        1
C2561        1
C4500        1
C2150        1
Name: CLASSIFICATION, Length: 71, dtype: int64

In [7]:
# You may find it helpful to look at CLASSIFICATION value counts >1
classification_counts[classification_counts > 1]

C1000    17326
C2000     6074
C1200     4837
C3000     1918
C2100     1883
C7000      777
C1700      287
C4000      194
C5000      116
C1270      114
C2700      104
C2800       95
C7100       75
C1300       58
C1280       50
C1230       36
C1400       34
C7200       32
C2300       32
C1240       30
C8000       20
C7120       18
C1500       16
C1800       15
C6000       15
C1250       14
C8200       11
C1238       10
C1278       10
C1235        9
C1237        9
C7210        7
C2400        6
C1720        6
C4100        6
C1257        5
C1600        5
C1260        3
C2710        3
C0           3
C3200        2
C1234        2
C1246        2
C1267        2
C1256        2
Name: CLASSIFICATION, dtype: int64

In [8]:
# Choose a cutoff value and create a list of classifications to be replaced
# use the variable name `classifications_to_replace`
classifications_to_replace = list(classification_counts[classification_counts < 1000].index)

# Replace in dataframe
for cls in classifications_to_replace:
    application_df['CLASSIFICATION'] = application_df['CLASSIFICATION'].replace(cls,"Other")
    
# Check to make sure binning was successful
application_df['CLASSIFICATION'].value_counts()

C1000    17326
C2000     6074
C1200     4837
Other     2261
C3000     1918
C2100     1883
Name: CLASSIFICATION, dtype: int64

In [9]:
# Convert categorical data to numeric with `pd.get_dummies`
application_with_dummies_df = pd.get_dummies(application_df)
print(application_with_dummies_df.shape)
application_with_dummies_df.columns

(34299, 41)


Index(['STATUS', 'ASK_AMT', 'IS_SUCCESSFUL', 'APPLICATION_TYPE_Other',
       'APPLICATION_TYPE_T19', 'APPLICATION_TYPE_T3', 'APPLICATION_TYPE_T4',
       'APPLICATION_TYPE_T5', 'APPLICATION_TYPE_T6',
       'AFFILIATION_CompanySponsored', 'AFFILIATION_Family/Parent',
       'AFFILIATION_Independent', 'AFFILIATION_National', 'AFFILIATION_Other',
       'AFFILIATION_Regional', 'CLASSIFICATION_C1000', 'CLASSIFICATION_C1200',
       'CLASSIFICATION_C2000', 'CLASSIFICATION_C2100', 'CLASSIFICATION_C3000',
       'CLASSIFICATION_Other', 'USE_CASE_CommunityServ', 'USE_CASE_Heathcare',
       'USE_CASE_Other', 'USE_CASE_Preservation', 'USE_CASE_ProductDev',
       'ORGANIZATION_Association', 'ORGANIZATION_Co-operative',
       'ORGANIZATION_Corporation', 'ORGANIZATION_Trust', 'INCOME_AMT_0',
       'INCOME_AMT_1-9999', 'INCOME_AMT_10000-24999',
       'INCOME_AMT_100000-499999', 'INCOME_AMT_10M-50M', 'INCOME_AMT_1M-5M',
       'INCOME_AMT_25000-99999', 'INCOME_AMT_50M+', 'INCOME_AMT_5M-10M',
 

In [10]:
# Split our preprocessed data into our features and target arrays
X = application_with_dummies_df.drop(["IS_SUCCESSFUL"], axis='columns').values
y = application_with_dummies_df["IS_SUCCESSFUL"].values

# Split the preprocessed data into a training and testing dataset
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=78)

In [11]:
# Create a StandardScaler instances
scaler = StandardScaler()

# Fit the StandardScaler
X_scaler = scaler.fit(X_train)

# Scale the data
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

## Compile, Train and Evaluate the Model

In [12]:
# Create a method that creates a new Sequential model with hyperparameter options
 # optimisation was performed by varying the number of layers between 1 and 6,
def create_model(hp):
    nn_model = tf.keras.models.Sequential()

    # Allow kerastuner to decide which activation function to use in hidden layers
    activation = hp.Choice('activation',['relu','tanh','sigmoid'])
    
    # Allow kerastuner to decide number of neurons in first layer
    nn_model.add(tf.keras.layers.Dense(units=hp.Int('first_units',
        min_value=1,
        max_value=120,
        step=5), activation=activation, input_dim=43))

    # Allow kerastuner to decide number of hidden layers and neurons in hidden layers
    for i in range(hp.Int('num_layers', 1, 6)):
        nn_model.add(tf.keras.layers.Dense(units=hp.Int('units_' + str(i),
            min_value=1,
            max_value=120,
            step=5),
            activation=activation))
    
    nn_model.add(tf.keras.layers.Dense(units=1, activation="sigmoid"))

    # Compile the model
    nn_model.compile(loss="binary_crossentropy", optimizer='adam', metrics=["accuracy"])
    
    return nn_model

In [13]:
# Import the kerastuner library

tuner = kt.Hyperband(
    create_model,
    objective="val_accuracy",
    max_epochs=20,
    hyperband_iterations=2)

Reloading Tuner from ./untitled_project/tuner0.json


In [14]:
# Run the kerastuner search for best hyperparameters
tuner.search(X_train_scaled,y_train,epochs=20,validation_data=(X_test_scaled,y_test))


Search: Running Trial #3

Value             |Best Value So Far |Hyperparameter
sigmoid           |relu              |activation
5                 |5                 |first_units
1                 |6                 |num_layers
3                 |7                 |units_0
7                 |1                 |units_1
9                 |1                 |units_2
3                 |1                 |units_3
3                 |1                 |units_4
7                 |1                 |units_5
3                 |3                 |tuner/epochs
0                 |0                 |tuner/initial_epoch
2                 |2                 |tuner/bracket
0                 |0                 |tuner/round

Epoch 1/3


Traceback (most recent call last):
  File "/Users/jody-annfrazer/anaconda3/lib/python3.10/site-packages/keras_tuner/src/engine/base_tuner.py", line 268, in _try_run_and_update_trial
    self._run_and_update_trial(trial, *fit_args, **fit_kwargs)
  File "/Users/jody-annfrazer/anaconda3/lib/python3.10/site-packages/keras_tuner/src/engine/base_tuner.py", line 233, in _run_and_update_trial
    results = self.run_trial(trial, *fit_args, **fit_kwargs)
  File "/Users/jody-annfrazer/anaconda3/lib/python3.10/site-packages/keras_tuner/src/tuners/hyperband.py", line 426, in run_trial
    return super().run_trial(trial, *fit_args, **fit_kwargs)
  File "/Users/jody-annfrazer/anaconda3/lib/python3.10/site-packages/keras_tuner/src/engine/tuner.py", line 307, in run_trial
    obj_value = self._build_and_fit_model(trial, *args, **copied_kwargs)
  File "/Users/jody-annfrazer/anaconda3/lib/python3.10/site-packages/keras_tuner/src/engine/tuner.py", line 227, in _build_and_fit_model
    results = self.hyper

RuntimeError: Number of consecutive failures exceeded the limit of 3.
Traceback (most recent call last):
  File "/Users/jody-annfrazer/anaconda3/lib/python3.10/site-packages/keras_tuner/src/engine/base_tuner.py", line 268, in _try_run_and_update_trial
    self._run_and_update_trial(trial, *fit_args, **fit_kwargs)
  File "/Users/jody-annfrazer/anaconda3/lib/python3.10/site-packages/keras_tuner/src/engine/base_tuner.py", line 233, in _run_and_update_trial
    results = self.run_trial(trial, *fit_args, **fit_kwargs)
  File "/Users/jody-annfrazer/anaconda3/lib/python3.10/site-packages/keras_tuner/src/tuners/hyperband.py", line 426, in run_trial
    return super().run_trial(trial, *fit_args, **fit_kwargs)
  File "/Users/jody-annfrazer/anaconda3/lib/python3.10/site-packages/keras_tuner/src/engine/tuner.py", line 307, in run_trial
    obj_value = self._build_and_fit_model(trial, *args, **copied_kwargs)
  File "/Users/jody-annfrazer/anaconda3/lib/python3.10/site-packages/keras_tuner/src/engine

In [15]:
# Get best model hyperparameters
best_hyper = tuner.get_best_hyperparameters(1)[0]
best_hyper.values

{'activation': 'relu',
 'first_units': 5,
 'num_layers': 6,
 'units_0': 7,
 'tuner/epochs': 3,
 'tuner/initial_epoch': 0,
 'tuner/bracket': 2,
 'tuner/round': 0,
 'units_1': 1,
 'units_2': 1,
 'units_3': 1,
 'units_4': 1,
 'units_5': 1}

In [16]:
# Evaluate the top 3 models against the test dataset
# top_model = tuner.get_best_models(3)
# for model in top_model:
    # model_loss, model_accuracy = model.evaluate(X_test_scaled,y_test,verbose=2)
    # print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

In [17]:
# Define the model - deep neural net, i.e., the number of input features and hidden nodes for each layer.
# Creating model based on the top hyperparameter
number_input_features = len(X_train[0])
hidden_nodes_layer1 =  21
hidden_nodes_layer2 = 106

nn = tf.keras.models.Sequential()

# First hidden layer
nn.add(tf.keras.layers.Dense(units=hidden_nodes_layer1, input_dim=number_input_features, activation="relu"))

# Second hidden layer
nn.add(tf.keras.layers.Dense(units=hidden_nodes_layer2, activation="relu"))

# Output layer
nn.add(tf.keras.layers.Dense(units=1, activation="sigmoid"))

# Check the structure of the model
nn.summary()

# Compile the model
nn.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])

# Train the model
fit_model = nn.fit(X_train_scaled, y_train, epochs=100)

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_3 (Dense)             (None, 21)                861       
                                                                 
 dense_4 (Dense)             (None, 106)               2332      
                                                                 
 dense_5 (Dense)             (None, 1)                 107       
                                                                 
Total params: 3300 (12.89 KB)
Trainable params: 3300 (12.89 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________
Epoch 1/100
804/804 [==============================] - 2s 1ms/step - loss: 0.5779 - accuracy: 0.7147
Epoch 2/100
804/804 [==============================] - 1s 906us/step - loss: 0.5604 - accuracy: 0.7285
Epoch 3/100
804/804 [==============================] - 1s 870us/step - loss: 0.5

804/804 [==============================] - 1s 931us/step - loss: 0.5416 - accuracy: 0.7392
Epoch 73/100
804/804 [==============================] - 1s 876us/step - loss: 0.5417 - accuracy: 0.7392
Epoch 74/100
804/804 [==============================] - 1s 872us/step - loss: 0.5414 - accuracy: 0.7397
Epoch 75/100
804/804 [==============================] - 1s 858us/step - loss: 0.5414 - accuracy: 0.7395
Epoch 76/100
804/804 [==============================] - 1s 919us/step - loss: 0.5416 - accuracy: 0.7383
Epoch 77/100
804/804 [==============================] - 1s 944us/step - loss: 0.5415 - accuracy: 0.7391
Epoch 78/100
804/804 [==============================] - 1s 1ms/step - loss: 0.5414 - accuracy: 0.7392
Epoch 79/100
804/804 [==============================] - 1s 1ms/step - loss: 0.5414 - accuracy: 0.7386
Epoch 80/100
804/804 [==============================] - 1s 955us/step - loss: 0.5413 - accuracy: 0.7395
Epoch 81/100
804/804 [==============================] - 1s 909us/step - loss: 0.5

In [19]:
# Evaluate the model using the test data
model_loss, model_accuracy = nn.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

268/268 - 0s - loss: 0.5618 - accuracy: 0.7256 - 178ms/epoch - 666us/step
Loss: 0.5617562532424927, Accuracy: 0.7255976796150208


In [20]:
# Changing the number of hidden layers 

In [21]:
# Define the model - deep neural net, i.e., the number of input features and hidden nodes for each layer.
number_input_features = len(X_train[0])
hidden_nodes_layer1 =  21
hidden_nodes_layer2 = 106
hidden_nodes_layer3 = 80
hidden_nodes_layer4 = 30
hidden_nodes_layer5 = 21


nn = tf.keras.models.Sequential()

# First hidden layer
nn.add(tf.keras.layers.Dense(units=hidden_nodes_layer1, input_dim=number_input_features, activation="relu"))

# Second hidden layer
nn.add(tf.keras.layers.Dense(units=hidden_nodes_layer2, activation="relu"))

# Third hidden layer
nn.add(tf.keras.layers.Dense(units=hidden_nodes_layer3, activation="relu"))

# Fourth hidden layer
nn.add(tf.keras.layers.Dense(units=hidden_nodes_layer4, activation="relu"))
# Fifth hidden layer
nn.add(tf.keras.layers.Dense(units=hidden_nodes_layer5, activation="relu"))

# Output layer
nn.add(tf.keras.layers.Dense(units=1, activation="sigmoid"))

# Check the structure of the model
nn.summary()

Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_6 (Dense)             (None, 21)                861       
                                                                 
 dense_7 (Dense)             (None, 106)               2332      
                                                                 
 dense_8 (Dense)             (None, 80)                8560      
                                                                 
 dense_9 (Dense)             (None, 30)                2430      
                                                                 
 dense_10 (Dense)            (None, 21)                651       
                                                                 
 dense_11 (Dense)            (None, 1)                 22        
                                                                 
Total params: 14856 (58.03 KB)
Trainable params: 14856

In [22]:
# Compile the model
nn.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])

In [23]:
# Train the model
fit_model = nn.fit(X_train_scaled, y_train, epochs=100)

Epoch 1/100
804/804 [==============================] - 2s 1ms/step - loss: 0.5759 - accuracy: 0.7210
Epoch 2/100
804/804 [==============================] - 1s 1ms/step - loss: 0.5592 - accuracy: 0.7306
Epoch 3/100
804/804 [==============================] - 1s 1ms/step - loss: 0.5554 - accuracy: 0.7308
Epoch 4/100
804/804 [==============================] - 1s 1ms/step - loss: 0.5546 - accuracy: 0.7313
Epoch 5/100
804/804 [==============================] - 1s 1ms/step - loss: 0.5527 - accuracy: 0.7338
Epoch 6/100
804/804 [==============================] - 1s 1ms/step - loss: 0.5524 - accuracy: 0.7341
Epoch 7/100
804/804 [==============================] - 1s 1ms/step - loss: 0.5516 - accuracy: 0.7345
Epoch 8/100
804/804 [==============================] - 1s 1ms/step - loss: 0.5510 - accuracy: 0.7348
Epoch 9/100
804/804 [==============================] - 1s 1ms/step - loss: 0.5502 - accuracy: 0.7352
Epoch 10/100
804/804 [==============================] - 1s 1ms/step - loss: 0.5493 - accura

804/804 [==============================] - 1s 2ms/step - loss: 0.5382 - accuracy: 0.7403
Epoch 82/100
804/804 [==============================] - 1s 1ms/step - loss: 0.5381 - accuracy: 0.7410
Epoch 83/100
804/804 [==============================] - 1s 1ms/step - loss: 0.5386 - accuracy: 0.7409
Epoch 84/100
804/804 [==============================] - 1s 1ms/step - loss: 0.5383 - accuracy: 0.7404
Epoch 85/100
804/804 [==============================] - 1s 1ms/step - loss: 0.5383 - accuracy: 0.7409
Epoch 86/100
804/804 [==============================] - 1s 1ms/step - loss: 0.5374 - accuracy: 0.7407
Epoch 87/100
804/804 [==============================] - 1s 1ms/step - loss: 0.5377 - accuracy: 0.7412
Epoch 88/100
804/804 [==============================] - 1s 1ms/step - loss: 0.5384 - accuracy: 0.7406
Epoch 89/100
804/804 [==============================] - 1s 2ms/step - loss: 0.5379 - accuracy: 0.7408
Epoch 90/100
804/804 [==============================] - 1s 1ms/step - loss: 0.5380 - accuracy: 

In [24]:
# Evaluate the model using the test data
model_loss, model_accuracy = nn.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

268/268 - 0s - loss: 0.5692 - accuracy: 0.7250 - 291ms/epoch - 1ms/step
Loss: 0.5692383646965027, Accuracy: 0.7250145673751831


In [25]:
# Export our model to HDF5 file
nn.save("AlphabetSoupCharity_Optimization.h5")

/Users/jody-annfrazer/anaconda3/lib/python3.10/site-packages/keras/src/engine/training.py:3000: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(
